In [199]:
import torch
from going_modular import going_modular
from pathlib import Path
import torch
import torchvision
import torchinfo
from torch import nn
from torchvision import transforms

In [203]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [204]:
#Transforms for images
transforms = torchvision.transforms.Compose([
    transforms.Resize(size=(64,64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))
    
])

In [205]:
#Dataset
import torchvision
from torch.utils.data import DataLoader
import os

data_path = Path('data/')

train_dir = data_path / "train"
test_dir = data_path / "test"

train_data = torchvision.datasets.CIFAR10(root = train_dir , train=True, download=True, transform=transforms)
test_data = torchvision.datasets.CIFAR10(root = test_dir, train=False, download=True)

train_dataloader = DataLoader(dataset=train_data, num_workers=os.cpu_count(), batch_size=128, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, num_workers=os.cpu_count(), batch_size=128, shuffle=True)


100%|██████████| 170498071/170498071 [00:34<00:00, 4907340.09it/s]


Extracting data/train/cifar-10-python.tar.gz to data/train


100%|██████████| 170498071/170498071 [00:38<00:00, 4382836.51it/s]


Extracting data/test/cifar-10-python.tar.gz to data/test


In [206]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        # print(m.weight.data)
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [4]:
# class weight_initialization(nn.Module):
#     def __init__(
#         self,
#         weights_std_deviation = 0.2
#     ):
        
#         super().__init__()
        
#         self.wi = nn.init.normal_(mean=0.0, std=weights_std_deviation)
        
        
#     def forward(self, x):
#         initialized_weights = self.wi(x)
#         return initialized_weights

In [207]:
class Generator(nn.Module):
    def __init__(
        self,
        latent_vector_size = 100,
        no_of_channels = 3,
        kernel_size = (4,4),
        stride: int = 2,
        number_of_feature_maps: int = 64,
        padding: int = 1,
    ):
        
        super().__init__()

        # self.wi = weight_initialization()
        self.main = nn.Sequential(
            
            
            
            nn.ConvTranspose2d(latent_vector_size, number_of_feature_maps * 16 , kernel_size=kernel_size, stride=stride, padding=0),
            nn.BatchNorm2d(number_of_feature_maps * 16),
            nn.ReLU(),
            
            #shape = (...,1024, 4, 4)
            nn.ConvTranspose2d(number_of_feature_maps * 16, number_of_feature_maps * 8 , kernel_size=kernel_size, stride=stride, padding=padding),
            nn.BatchNorm2d(number_of_feature_maps * 8),
            nn.ReLU(),
            
            #shape = (..., 512, 8, 8)
            nn.ConvTranspose2d(number_of_feature_maps * 8, number_of_feature_maps * 4 , kernel_size=kernel_size, stride=stride, padding=padding),
            nn.ReLU(),
            nn.BatchNorm2d(number_of_feature_maps * 4),
            
             #shape = (..., 256, 16, 16)
            nn.ConvTranspose2d(number_of_feature_maps * 4, number_of_feature_maps * 2 , kernel_size=kernel_size, stride=stride, padding=padding),
            nn.ReLU(),
            nn.BatchNorm2d(number_of_feature_maps * 2),
            
             #shape = (..., 128, 32, 32)
            nn.ConvTranspose2d(number_of_feature_maps * 2, no_of_channels , kernel_size=kernel_size, stride=stride, padding=padding),
            nn.Tanh()
            #shape = (..., 3, 64, 64)
        )
        
    def forward(self, x):
        x = self.main(x)
        return x

In [208]:
#Intializing the Generator instance
generator = Generator().to(device)

#Applying the weights transformation
generator.apply(weights_init)

#Printing the structure
print(generator)

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(2, 2))
    (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): ReLU()
    (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (10): ReLU()
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ConvTranspose2d(128, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (13): Tanh()
  )
)


In [209]:
from torchinfo import summary


summary(model=generator,
        input_size=(128, 100, 1, 1),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
Generator (Generator)                    [128, 100, 1, 1]     [128, 3, 64, 64]     --                   True
├─Sequential (main)                      [128, 100, 1, 1]     [128, 3, 64, 64]     --                   True
│    └─ConvTranspose2d (0)               [128, 100, 1, 1]     [128, 1024, 4, 4]    1,639,424            True
│    └─BatchNorm2d (1)                   [128, 1024, 4, 4]    [128, 1024, 4, 4]    2,048                True
│    └─ReLU (2)                          [128, 1024, 4, 4]    [128, 1024, 4, 4]    --                   --
│    └─ConvTranspose2d (3)               [128, 1024, 4, 4]    [128, 512, 8, 8]     8,389,120            True
│    └─BatchNorm2d (4)                   [128, 512, 8, 8]     [128, 512, 8, 8]     1,024                True
│    └─ReLU (5)                          [128, 512, 8, 8]     [128, 512, 8, 8]     --                   --
│    └─ConvTranspo

In [222]:
class Discriminator(nn.Module):
    def __init__(
        self,
        no_of_channels = 3,
        kernel_size = (4,4),
        stride: int = 2,
        number_of_feature_maps: int = 64,
        padding: int = 1,
        lr_slope=0.2,
        # latent_vector_size: int = 100
    ):
        
        super().__init__()
        
         #shape = (..., 3, 64, 64)
        self.main = nn.Sequential(
            nn.Conv2d(no_of_channels, number_of_feature_maps * 2 , kernel_size=kernel_size, stride=stride, padding=padding),
            # nn.BatchNorm2d(number_of_feature_maps * 8),
            nn.LeakyReLU(negative_slope=lr_slope),
                
                #shape = (...,1024, 32, 32)
            nn.Conv2d(number_of_feature_maps * 2, number_of_feature_maps * 4 , kernel_size=kernel_size, stride=stride, padding=padding),
            nn.BatchNorm2d(number_of_feature_maps * 4),
            nn.LeakyReLU(negative_slope=lr_slope),
                
                #shape = (..., 512, 16, 16)
            nn.Conv2d(number_of_feature_maps * 4, number_of_feature_maps * 8 , kernel_size=kernel_size, stride=stride, padding=padding),
            nn.BatchNorm2d(number_of_feature_maps * 8),
            nn.LeakyReLU(negative_slope=lr_slope),
                
                #shape = (..., 256, 8, 8)
            nn.Conv2d(number_of_feature_maps * 8, number_of_feature_maps * 16 , kernel_size=kernel_size, stride=stride, padding=padding),
            nn.BatchNorm2d(number_of_feature_maps * 16),
            # nn.LeakyReLU(negative_slope=lr_slope),
            #  shape = (..., 128, 4, 4)
            
            # nn.Conv2d(number_of_feature_maps * 16, latent_vector_size , kernel_size=kernel_size, stride=stride, padding=padding),
            # nn.BatchNorm2d(latent_vector_size),
            nn.Sigmoid(),
         )
        
    def forward(self, x):
        return self.main(x)

In [223]:
#Intializing the Discriminator instance
discriminator = Discriminator().to(device)
discriminator = discriminator.apply(weights_init)
#Printing the structure
print(discriminator)

Discriminator(
  (main): Sequential(
    (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2)
    (5): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (6): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2)
    (8): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (9): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): Sigmoid()
  )
)


In [224]:
from torchinfo import summary


summary(model=discriminator,
        input_size=(128, 3, 64, 64),
        col_names=["input_size", "output_size", "num_params", "trainable"],
        col_width=20,
        row_settings=["var_names"])

Layer (type (var_name))                  Input Shape          Output Shape         Param #              Trainable
Discriminator (Discriminator)            [128, 3, 64, 64]     [128, 1024, 4, 4]    --                   True
├─Sequential (main)                      [128, 3, 64, 64]     [128, 1024, 4, 4]    --                   True
│    └─Conv2d (0)                        [128, 3, 64, 64]     [128, 128, 32, 32]   6,272                True
│    └─LeakyReLU (1)                     [128, 128, 32, 32]   [128, 128, 32, 32]   --                   --
│    └─Conv2d (2)                        [128, 128, 32, 32]   [128, 256, 16, 16]   524,544              True
│    └─BatchNorm2d (3)                   [128, 256, 16, 16]   [128, 256, 16, 16]   512                  True
│    └─LeakyReLU (4)                     [128, 256, 16, 16]   [128, 256, 16, 16]   --                   --
│    └─Conv2d (5)                        [128, 256, 16, 16]   [128, 512, 8, 8]     2,097,664            True
│    └─BatchNorm2d

In [154]:
class DCGAN(nn.Module):
    def __init__(
        self,
        no_of_channels = 3,
        kernel_size = (4,4),
        stride: int = 2,
        number_of_feature_maps: int = 64,
        padding: int = 1,
        lr_slope=0.2,
        latent_vector_size = 100,
    ):
        
        super().__init__()
        
        self.generator = Generator(latent_vector_size=latent_vector_size,no_of_channels=no_of_channels,kernel_size=kernel_size,stride=stride, number_of_feature_maps=number_of_feature_maps, padding=padding)
        self.discriminator = Discriminator(no_of_channels=no_of_channels, kernel_size=kernel_size, stride=stride, number_of_feature_maps=number_of_feature_maps,padding=padding, lr_slope=lr_slope)
        
        
    def forward(self, x):
        return self.generator(self.discriminator(x))

In [155]:
#Intializing the DCGAN
dcgan = DCGAN()
dcgan.to(device)
print(dcgan)

DCGAN(
  (generator): Generator(
    (main): Sequential(
      (0): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(2, 2))
      (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (7): ReLU()
      (8): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (9): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (10): ReLU()
      (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (12): ConvTranspose2d(128, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (13): Tanh()
    )
  )
  (discriminator): Discriminator(
    (main): 

In [212]:
epochs = 50
beta_1 = 0.5
lr_optimizer = 0.0002
loss_fn = nn.BCELoss()  #BCELoss function

optimizerD = torch.optim.Adam(params=discriminator.parameters(), betas=(beta_1, 0.999), lr=lr_optimizer) #For discriminator
optimizerG = torch.optim.Adam(params=generator.parameters(), betas=(beta_1, 0.999), lr=lr_optimizer) #For generator

batch_size = 128
latent_vector_size = 100

real_label = 1
fake_label = 0


loss_g = []
loss_d = []
img_list = []

# Fixed noise for generating the images
fixed_noise = torch.randn((batch_size, latent_vector_size, 1, 1), dtype=torch.float32, device=device)

In [168]:
for batch, (X, y) in enumerate(train_dataloader, 0):
    print(batch, X, y)
    break

0 tensor([[[[-0.5059, -0.5529, -0.6000,  ..., -0.8510, -0.8039, -0.8118],
          [-0.6078, -0.7333, -0.7490,  ..., -0.7098, -0.5294, -0.5216],
          [-0.2471, -0.2627, -0.3490,  ..., -0.6471, -0.5451, -0.4588],
          ...,
          [-0.9608, -0.9608, -0.9765,  ..., -0.9765, -0.9608, -0.9451],
          [-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
          [-0.2078, -0.2471, -0.2471,  ..., -0.2314, -0.2471, -0.2235]],

         [[-0.0196, -0.0902, -0.1294,  ..., -0.2863, -0.2392, -0.2471],
          [-0.0824, -0.2078, -0.2235,  ..., -0.1529, -0.0353, -0.0196],
          [ 0.1529,  0.1294,  0.0431,  ..., -0.0667,  0.0275,  0.0588],
          ...,
          [-0.3098, -0.3333, -0.3647,  ..., -0.3882, -0.2941, -0.2941],
          [-0.4980, -0.4118, -0.4039,  ..., -0.5059, -0.4667, -0.4745],
          [ 0.0196, -0.0039,  0.0196,  ..., -0.0118, -0.0039,  0.0196]],

         [[-0.7255, -0.7490, -0.7255,  ..., -0.8588, -0.7804, -0.7098],
          [-0.7255, -0.7804,

In [171]:
label = torch.full((b_size,), real_label, dtype=torch.float, device=device)
label.fill_(fake_label)
label

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])

In [167]:
real_cpu = data[0].to(device)
b_size = real_cpu.size(0)
b_size,  real_cpu

(128,
 tensor([[[[-0.0980, -0.0824, -0.0196,  ...,  0.0039, -0.0118, -0.0353],
           [-0.0980, -0.0588, -0.0039,  ...,  0.0039, -0.0196, -0.0196],
           [-0.1529, -0.0667, -0.0039,  ...,  0.0196,  0.0039, -0.0196],
           ...,
           [-0.1608, -0.1373, -0.1294,  ...,  0.0824,  0.0980, -0.0039],
           [-0.1608, -0.1529, -0.0667,  ...,  0.0745,  0.1451, -0.0039],
           [-0.1451, -0.1451, -0.1059,  ..., -0.0039,  0.1608,  0.0039]],
 
          [[-0.0745, -0.0667, -0.0196,  ...,  0.0275,  0.0196,  0.0275],
           [-0.0902, -0.0588, -0.0039,  ...,  0.0275,  0.0118,  0.0353],
           [-0.1137, -0.0588, -0.0039,  ...,  0.0431,  0.0353,  0.0275],
           ...,
           [-0.0980, -0.0824, -0.0745,  ...,  0.0824,  0.0824,  0.0118],
           [-0.0980, -0.0980, -0.0275,  ...,  0.0510,  0.0745,  0.0196],
           [-0.0824, -0.0824, -0.0588,  ..., -0.0118,  0.0431,  0.0118]],
 
          [[-0.0431, -0.0431, -0.0039,  ...,  0.0824,  0.0824,  0.0980],
       

In [218]:
#Training loop


generator.train()
discriminator.train()

for epoch in range(epochs):
    for batch, (X, y) in enumerate(train_dataloader):
        X, y = X.to(device), y.to(device)
        
        #Train the discriminator (with real data)
        
        ############################
        # (1) Update D network: maximize: log(1 - D(G(z)))
        ###########################
        batch_size = X.shape[0]

        real_data = torch.ones((batch_size,), device=device, dtype=torch.float32)
        
        # 1. Forward pass
        y_pred = discriminator(X)
        print(y_pred)
        
        # 2. Calculate  and accumulate loss
        loss_real = loss_fn(y_pred, real_data)
        # loss_d.append(loss.item())

        # 3. Optimizer zero grad
        optimizerD.zero_grad()

        # 4. Loss backward
        loss_real.backward()

        # 5. Optimizer step
        optimizerD.step()
        
        
        #Train the discriminator (with fake data)
        
        noise = torch.randn((batch_size, latent_vector_size, 1, 1), device=device)
        fake_data = torch.zeros((batch_size,), device=device, dtype=torch.float32)
        noise_generated_by_generator = generator(noise)
        
        #1. Forward pass
        y_pred = discriminator(noise_generated_by_generator)

        # 2. Calculate  and accumulate loss
        loss_fake = loss_fn(y_pred, fake_data)
        # loss_d.append(loss.item())

        # 3. Optimizer zero grad
        optimizerD.zero_grad()

        # 4. Loss backward
        loss_fake.backward()

        # 5. Optimizer step
        optimizerD.step()
        
        #Accumulating total discriminator loss
        loss_d.append(loss_real + loss_fake)
        
        
        
        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
       
        labels = torch.ones((batch_size,), device=device, dtype=torch.float32)
        
        #q. Forward pass
        y_pred = discriminator(noise_generated_by_generator)
        
        #2. Calculate and accumulate loss
        loss = loss_fn(y_pred,labels)
        
        
        # 3. Optimizer zero grad
        optimizerG.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizerG.step()
        
        loss_g.append(loss.item())
        
        if epoch % 10 == 0:
            print("Epoch: ", epoch, "Generator loss: ", loss_g, "Discriminator loss: ", loss_d)
            
    if (epoch % 10 == 0) or ((epoch == epoch - 1) and (i == len(train_dataloader)-1)):
        
            generator.eval()
            with torch.inference_mode():
                fake = generator(fixed_noise).detach().cpu()
            img_list.append(torchvision.utils.make_grid(fake, padding=2, normalize=True))

tensor([[[[0.1950, 0.2300, 0.5285, 0.5896],
          [0.5716, 0.6724, 0.7499, 0.5886],
          [0.5728, 0.4234, 0.7107, 0.6535],
          [0.3770, 0.5665, 0.7673, 0.5450]],

         [[0.7160, 0.4667, 0.7805, 0.4853],
          [0.7144, 0.6089, 0.8224, 0.3188],
          [0.6232, 0.3933, 0.4965, 0.4608],
          [0.4991, 0.8492, 0.7735, 0.8155]],

         [[0.7492, 0.7031, 0.6195, 0.5004],
          [0.1612, 0.2214, 0.1322, 0.3616],
          [0.4305, 0.4333, 0.5556, 0.5596],
          [0.7225, 0.6792, 0.5900, 0.5882]],

         ...,

         [[0.5153, 0.4766, 0.7324, 0.6432],
          [0.4094, 0.5450, 0.7197, 0.6978],
          [0.4342, 0.7949, 0.3640, 0.4233],
          [0.2000, 0.7110, 0.1713, 0.3191]],

         [[0.3699, 0.3325, 0.3186, 0.4511],
          [0.6182, 0.4321, 0.7198, 0.0941],
          [0.3208, 0.1823, 0.6420, 0.3315],
          [0.5116, 0.7489, 0.3575, 0.2514]],

         [[0.6837, 0.6520, 0.7078, 0.3880],
          [0.3055, 0.8798, 0.6604, 0.4107],
       

ValueError: Using a target size (torch.Size([128])) that is different to the input size (torch.Size([128, 1024, 4, 4])) is deprecated. Please ensure they have the same size.

In [ ]:
#Visulizing the losses

import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(loss_g,label="G")
plt.plot(loss_d,label="D")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()